In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data_cleaned"
                 ".txt", sep="|", low_memory=False)

In [3]:
df.columns

Index(['Identifiant de document', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain'],
      dtype='object')

In [4]:
df["Identifiant de document"].value_counts(dropna=False)

Identifiant de document
NaN    1532718
Name: count, dtype: int64

In [5]:
cols_to_drop = df.columns[df.isna().sum() == df.shape[0]].to_list()
cols_to_drop

['Identifiant de document',
 'Reference document',
 '1 Articles CGI',
 '2 Articles CGI',
 '3 Articles CGI',
 '4 Articles CGI',
 '5 Articles CGI',
 'Identifiant local']

In [6]:
cols_to_create = df.columns[df.isna().sum() != df.shape[0]].to_list()

In [7]:
import mysql.connector

In [8]:
print(cols_to_create)

['No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere', 'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal', 'Commune', 'Code departement', 'Code commune', 'Prefixe de section', 'Section', 'No plan', 'No Volume', '1er lot', 'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot', '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot', 'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot', 'Nombre de lots', 'Code type local', 'Type local', 'Surface reelle bati', 'Nombre pieces principales', 'Nature culture', 'Nature culture speciale', 'Surface terrain']


In [9]:
df['Code type local'].dropna()

0          3.0
1          3.0
2          2.0
3          1.0
4          3.0
          ... 
1532713    2.0
1532714    3.0
1532715    2.0
1532716    2.0
1532717    4.0
Name: Code type local, Length: 909840, dtype: float64

In [10]:
df[cols_to_create].dtypes

No disposition                  int64
Date mutation                  object
Nature mutation                object
Valeur fonciere               float64
No voie                       float64
B/T/Q                          object
Type de voie                   object
Code voie                      object
Voie                           object
Code postal                   float64
Commune                        object
Code departement               object
Code commune                    int64
Prefixe de section            float64
Section                        object
No plan                         int64
No Volume                      object
1er lot                        object
Surface Carrez du 1er lot     float64
2eme lot                       object
Surface Carrez du 2eme lot    float64
3eme lot                       object
Surface Carrez du 3eme lot    float64
4eme lot                      float64
Surface Carrez du 4eme lot    float64
5eme lot                       object
Surface Carr

In [11]:
df["No Volume"].value_counts()

No Volume
2        746
1        738
3        250
4        158
5        114
        ... 
11100      1
11200      1
11301      1
11306      1
1148       1
Name: count, Length: 337, dtype: int64

In [ ]:
host = "localhost"
user = "root"
password = "root"

try:
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
    )

    if connection.is_connected():
        print(f"Connected to MySQL server (version {connection.get_server_info()})")

except mysql.connector.Error as e:
    print(f"Error connecting to MySQL: {e}")

else:
    cursor = connection.cursor()
    cursor.execute("CREATE DATABASE IF NOT EXISTS dvf")
    cursor.execute("USE dvf")
    cursor.execute("DROP TABLE IF EXISTS data")
    query = """
        CREATE TABLE data (
            id INT AUTO_INCREMENT PRIMARY KEY,
            no_disposition INT,
            date_mutation VARCHAR(30),
            nature_mutation VARCHAR(100),
            valeur_fonciere DECIMAL,
            no_voie DECIMAL,
            btq VARCHAR(30),
            type_voie VARCHAR(30),
            code_voie VARCHAR(30),
            voie VARCHAR(30),
            code_postal VARCHAR(30),
            commune VARCHAR(30),
            code_departement VARCHAR(30),
            code_commune VARCHAR(30),
            prefixe_section VARCHAR(30),
            section VARCHAR(30),
            no_plan INT,
            no_volume VARCHAR(30),
            lot_1 VARCHAR(30),
            carrez_lot_1 DECIMAL,
            lot_2 VARCHAR(30),
            carrez_lot_2 DECIMAL,
            lot_3 VARCHAR(30),
            carrez_lot_3 DECIMAL,
            lot_4 VARCHAR(30),
            carrez_lot_4 DECIMAL,
            lot_5 VARCHAR(30),
            carrez_lot_5 DECIMAL,
            nombre_lots INT,
            code_type_local DECIMAL,
            type_local VARCHAR(100),
            surface_reelle_bati DECIMAL,
            nombre_pieces_principales DECIMAL,
            nature_culture VARCHAR(30),
            nature_culture_speciale VARCHAR(30),
            surface_terrain DECIMAL
        )
    """
    cursor.execute(query)
    
    cursor.execute("DESCRIBE data")
    columns = [cols[0] for cols in cursor.fetchall()][1:]
    
    for _, record in df[cols_to_create].iterrows():
        insert_query = f"""
            INSERT INTO data (
                {", ".join(columns)}
            ) VALUES (
                {", ".join(["%s"]*len(columns))}
            )
        """
        record.replace(np.nan, None, inplace=True)
        values = tuple(record.values)
        cursor.execute(insert_query, values)
    connection.commit()
    
finally:
    if connection.is_connected():
        connection.close()
        print("Connection closed")


Connected to MySQL server (version 8.2.0)


In [13]:
# df["Nature mutation"].value_counts()